In [1]:
!pip install -upgrade -quiet langchain_core langchain-community langchain-openai
!pip install openai
!pip install langchain
!pip install langchain_openai


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
#튜플이나 리스트같은 '컬렉션타입' 데이터에서 원하는 인덱스의 요소를 추출하는 데 사용
from operator import itemgetter

#러너블 클래스 : 함수를 감싸서 체인화할 수 있게 해줌
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

#Output(결과물)을 content만 가져올 수 있도록 변환
#choices[0].content... <- {model,,,}
from langchain_core.output_parsers import StrOutputParser

#특정한 대화 양식(템플릿)을 정의하여 쉽게 질의응답할 수 있도록 해줌
#우리가 나눈 대화를 관리하는 데 사용
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#openai와의 대화형 질의응답
from langchain_openai import ChatOpenAI

#openai와 나눈 대화를 저장하고 관리하는 데 사용하는 클래스
#'이전 대화를 기반으로' 현재의 대화를 이끌어나가도록 함
from langchain.memory import ConversationBufferWindowMemory

In [5]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPEN_AI_KEY")


In [6]:
system_prompt = """
너는 나랑 친구인 상담사야.
내가 물어보는 질문에 대해서 까칠하게 알려줘야 해.
한국어로 답변해줘.
"""

In [20]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name='history'),
    ("human", "{user_input}")
])

#k=기억할 대화 쌍의 개수(보낸 메시지가 아니라 왔다갔다 한 대화 쌍의 개수)
#총 3개의 이전 대화 쌍을 기억함
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

In [21]:
chat_model = ChatOpenAI()
output_parser = StrOutputParser()

In [22]:
chain = ({"user_input":RunnablePassthrough()} |
        RunnablePassthrough.assign(history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"))
        | chat_prompt_template
        | chat_model
        | output_parser)

In [23]:
def load_memory(_):
  return memory.load_memory_variables(_)['history']

In [24]:
def chat_with_user(user_message):
  ai_message = chain.invoke(user_message)

  memory.save_context({"input":user_message}, {"output":ai_message})
  print(memory.load_memory_variables({}))

  return ai_message

In [33]:
chat_with_user("안녕! intj와 대화를 시작하고 싶은데 어떻게 말해야 할까?")

{'history': [HumanMessage(content='안녕! intj에게 만나자고 하고 싶은데 어떻게 말해야 할까?'), AIMessage(content='INTJ 유형은 보통 직설적이고 목적 지향적인 대화를 선호해. 그래서 간단하고 명확하게 "만나자!" 라고 말하는 것이 좋을 거야. 또한 만날 날짜와 시간을 제시하여 구체적으로 계획을 잡는 것도 도움이 될 거야. 그들은 일정을 미리 계획하고 준비하는 것을 선호하기 때문에 명확한 제안을 통해 만남을 주선하는 것이 INTJ에게 좋은 방법이 될 거야.'), HumanMessage(content='안녕! intj와 대화를 시작하고 싶은데 어떻게 말해야 할까?'), AIMessage(content='INTJ와 대화를 시작하고 싶다면 그들의 관심사나 전문 지식에 대해 물어보는 것이 좋을 수 있어. INTJ는 일반적으로 논리적이고 직설적인 대화를 선호하기 때문에, 관심 있는 주제에 대해 깊이 있는 대화를 나누는 것이 좋은 방법일 거야. 또한 그들의 의견이나 아이디어를 존중하고 인정해주는 것도 INTJ와의 대화를 시작하는 데 도움이 될 수 있어. 간단하게 "너의 의견이 궁금한데, 이 주제에 대해 어떻게 생각해?"와 같이 시작하는 것이 좋을 거야.'), HumanMessage(content='안녕! intj와 대화를 시작하고 싶은데 어떻게 말해야 할까?'), AIMessage(content='"안녕, 최근에 읽은 책/본 영화/들은 새로운 정보가 있어. 너의 의견이 궁금해. 같이 이야기해볼까?"와 같이 시작하는 것이 좋을 것 같아. INTJ는 새로운 정보나 아이디어에 관심을 가지는 경우가 많아서, 그들의 관심사나 지식에 대해 이야기하며 대화를 시작하는 것이 좋을 거야. 또한 논리적이고 목적 지향적인 대화를 선호하기 때문에, 구체적이고 관련성 있는 주제를 제시하여 대화를 이끌어가는 것이 INTJ와의 대화를 즐겁게 이어나갈 수 있는 방법이 될 거야.')]}


'"안녕, 최근에 읽은 책/본 영화/들은 새로운 정보가 있어. 너의 의견이 궁금해. 같이 이야기해볼까?"와 같이 시작하는 것이 좋을 것 같아. INTJ는 새로운 정보나 아이디어에 관심을 가지는 경우가 많아서, 그들의 관심사나 지식에 대해 이야기하며 대화를 시작하는 것이 좋을 거야. 또한 논리적이고 목적 지향적인 대화를 선호하기 때문에, 구체적이고 관련성 있는 주제를 제시하여 대화를 이끌어가는 것이 INTJ와의 대화를 즐겁게 이어나갈 수 있는 방법이 될 거야.'

In [35]:
while True:
  user_message = input("USER >> ")
  if user_message == 'quit':
    break
  ai_message = chat_with_user(user_message)
  print(f"AI >> {ai_message}")


USER >> 안녕! 내 이름은 조조
{'history': [HumanMessage(content='대화를 종료하려면 어떻게 해?'), AIMessage(content='대화를 종료하고 싶다면 간단히 "종료"라고 말해주면 돼. 그러면 대화가 종료되고 나는 아무런 대답을 하지 않을 거야. 언제든지 편하게 종료하고 싶을 때 말해줘. 함께 대화를 나누는 동안에는 네 마음을 존중해주는 것이 가장 중요하니까. 언제든지 다시 도움이 필요하거나 대화를 이어가고 싶을 때는 언제든지 나에게 다시 말을 걸어줘. 너를 지지하고 응원해주는 나는 항상 여기 있을 거야. 함께 대화를 나누어주셔서 고마워!'), HumanMessage(content='종료'), AIMessage(content='대화를 종료하셨군요. 언제든지 다시 도움이 필요하거나 이야기를 나누고 싶을 때는 언제든지 돌아와주시기 바랍니다. 즐거운 하루 보내시길 바랍니다! 종료됐습니다. 🌟👋'), HumanMessage(content='안녕! 내 이름은 조조'), AIMessage(content='안녕, 조조! 만나서 반가워. 어떤 이야기를 해주고 싶은 거니, 조조? 무엇이든 물어봐도 괜찮아. 함께 대화를 즐기면서 도와드릴게. 😊🌼')]}
AI >> 안녕, 조조! 만나서 반가워. 어떤 이야기를 해주고 싶은 거니, 조조? 무엇이든 물어봐도 괜찮아. 함께 대화를 즐기면서 도와드릴게. 😊🌼
USER >> 네 이름은 우주
{'history': [HumanMessage(content='종료'), AIMessage(content='대화를 종료하셨군요. 언제든지 다시 도움이 필요하거나 이야기를 나누고 싶을 때는 언제든지 돌아와주시기 바랍니다. 즐거운 하루 보내시길 바랍니다! 종료됐습니다. 🌟👋'), HumanMessage(content='안녕! 내 이름은 조조'), AIMessage(content='안녕, 조조! 만나서 반가워. 어떤 이야기를 해주고 싶은 거니, 조조? 무엇이든 물어봐도 괜찮아. 함께 대화를 즐기면서 도와드릴게. 😊🌼'

In [ ]:
##그라디오를 활용한 챗봇 껍데기 만들기

In [36]:
!pip install gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=bd526b13

In [41]:
import gradio as gr

import time

def response(user_message, chat_view):
  ai_message = chat_with_user(user_message)
  chat_view.append((user_message, ai_message))

  time.sleep(0.5)

  return"", chat_view

with gr.Blocks() as demo:
  chat_view = gr.Chatbot(label = "대나무숲")

  user_textbox = gr.Textbox(label='하고 싶은 말을 입력해주세요')

  user_textbox.submit(response, [user_textbox, chat_view], [user_textbox, chat_view])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8f32b81fdc0e187bfa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
